# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2022 9:00AM,237808,18,CLRSPC-20975466,"ClearSpec, LLC",Released
1,May 23 2022 8:59AM,237806,15,PNC356395C,"Person & Covey, Inc.",Released
2,May 23 2022 8:59AM,237806,15,PNC360131A,"Person & Covey, Inc.",Released
3,May 23 2022 8:59AM,237806,15,PNC360132A,"Person & Covey, Inc.",Released
4,May 23 2022 8:59AM,237806,15,PNC360133A,"Person & Covey, Inc.",Released
5,May 23 2022 8:59AM,237806,15,PNC360136A,"Person & Covey, Inc.",Released
6,May 23 2022 8:59AM,237806,15,PNC360140A,"Person & Covey, Inc.",Released
7,May 23 2022 8:59AM,237806,15,PNC360141A,"Person & Covey, Inc.",Released
8,May 23 2022 8:59AM,237806,15,PNC360142A,"Person & Covey, Inc.",Released
9,May 23 2022 8:59AM,237806,15,PNC360143A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,237801,Released,1
15,237804,Released,19
16,237805,Released,14
17,237806,Released,14
18,237808,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237801,NaN,NaN,1.0
237804,NaN,NaN,19.0
237805,NaN,NaN,14.0
237806,NaN,NaN,14.0
237808,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237774,1.0,0.0,33.0
237781,0.0,0.0,51.0
237787,0.0,0.0,1.0
237789,0.0,4.0,3.0
237792,0.0,0.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237774,1,0,33
237781,0,0,51
237787,0,0,1
237789,0,4,3
237792,0,0,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237774,1,0,33
1,237781,0,0,51
2,237787,0,0,1
3,237789,0,4,3
4,237792,0,0,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237774,1,,33
1,237781,,,51
2,237787,,,1
3,237789,,4,3
4,237792,,,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2022 9:00AM,237808,18,"ClearSpec, LLC"
1,May 23 2022 8:59AM,237806,15,"Person & Covey, Inc."
15,May 23 2022 8:58AM,237781,10,ISDIN Corporation
66,May 23 2022 8:54AM,237801,15,"Mizner Bioscience, LLC"
67,May 23 2022 8:54AM,237800,18,Hush Hush
78,May 23 2022 8:53AM,237799,18,Hush Hush
93,May 23 2022 8:53AM,237798,18,Hush Hush
113,May 23 2022 8:52AM,237796,18,Hush Hush
114,May 23 2022 8:50AM,237805,10,Beach Products Inc
128,May 23 2022 8:47AM,237804,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2022 9:00AM,237808,18,"ClearSpec, LLC",,,1
1,May 23 2022 8:59AM,237806,15,"Person & Covey, Inc.",,,14
2,May 23 2022 8:58AM,237781,10,ISDIN Corporation,,,51
3,May 23 2022 8:54AM,237801,15,"Mizner Bioscience, LLC",,,1
4,May 23 2022 8:54AM,237800,18,Hush Hush,,,11
5,May 23 2022 8:53AM,237799,18,Hush Hush,,,15
6,May 23 2022 8:53AM,237798,18,Hush Hush,,,20
7,May 23 2022 8:52AM,237796,18,Hush Hush,,,1
8,May 23 2022 8:50AM,237805,10,Beach Products Inc,,,14
9,May 23 2022 8:47AM,237804,10,ISDIN Corporation,,,19


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:00AM,237808,18,"ClearSpec, LLC",1,,
1,May 23 2022 8:59AM,237806,15,"Person & Covey, Inc.",14,,
2,May 23 2022 8:58AM,237781,10,ISDIN Corporation,51,,
3,May 23 2022 8:54AM,237801,15,"Mizner Bioscience, LLC",1,,
4,May 23 2022 8:54AM,237800,18,Hush Hush,11,,
5,May 23 2022 8:53AM,237799,18,Hush Hush,15,,
6,May 23 2022 8:53AM,237798,18,Hush Hush,20,,
7,May 23 2022 8:52AM,237796,18,Hush Hush,1,,
8,May 23 2022 8:50AM,237805,10,Beach Products Inc,14,,
9,May 23 2022 8:47AM,237804,10,ISDIN Corporation,19,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:00AM,237808,18,"ClearSpec, LLC",1,,
1,May 23 2022 8:59AM,237806,15,"Person & Covey, Inc.",14,,
2,May 23 2022 8:58AM,237781,10,ISDIN Corporation,51,,
3,May 23 2022 8:54AM,237801,15,"Mizner Bioscience, LLC",1,,
4,May 23 2022 8:54AM,237800,18,Hush Hush,11,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:00AM,237808,18,"ClearSpec, LLC",1,NaN,NaN
1,May 23 2022 8:59AM,237806,15,"Person & Covey, Inc.",14,NaN,NaN
2,May 23 2022 8:58AM,237781,10,ISDIN Corporation,51,NaN,NaN
3,May 23 2022 8:54AM,237801,15,"Mizner Bioscience, LLC",1,NaN,NaN
4,May 23 2022 8:54AM,237800,18,Hush Hush,11,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:00AM,237808,18,"ClearSpec, LLC",1,0.0,0.0
1,May 23 2022 8:59AM,237806,15,"Person & Covey, Inc.",14,0.0,0.0
2,May 23 2022 8:58AM,237781,10,ISDIN Corporation,51,0.0,0.0
3,May 23 2022 8:54AM,237801,15,"Mizner Bioscience, LLC",1,0.0,0.0
4,May 23 2022 8:54AM,237800,18,Hush Hush,11,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,951164,117,0.0,1.0
15,1188990,64,0.0,0.0
18,1426796,49,0.0,0.0
19,237789,3,4.0,0.0
22,237787,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,951164,117,0.0,1.0
1,15,1188990,64,0.0,0.0
2,18,1426796,49,0.0,0.0
3,19,237789,3,4.0,0.0
4,22,237787,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,117,0.0,1.0
1,15,64,0.0,0.0
2,18,49,0.0,0.0
3,19,3,4.0,0.0
4,22,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,117.0
1,15,Released,64.0
2,18,Released,49.0
3,19,Released,3.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,22
Status,,,,,
Completed,1.0,0.0,0.0,0.0,0.0
Executing,0.0,0.0,0.0,4.0,0.0
Released,117.0,64.0,49.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,22
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,4.0,0.0
2,Released,117.0,64.0,49.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,22
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,4.0,0.0
2,Released,117.0,64.0,49.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()